In [66]:
import os
import pandas as pd
import json

from model import num2cate_fit, num2cate_transform, generate_samples, generate_model_samples, ModelGene, findKeyAttrs, FindGroups, get_numAttrs, find_rules
from model.samples import DataGene
from model.data_encoder import DataEncoder
from joblib import dump, load

import numpy as np

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score

store_path = '../../front/src/testdata/'
dataset_name = 'adult'
model_name = 'knn'
theta = 0.8


In [70]:
dataset_name='academic'
dataset_path = '../server/{}.csv'.format(dataset_name)
data = pd.read_csv(dataset_path)
mdlp = num2cate_fit(data)
key_attrs = findKeyAttrs(num2cate_transform(data, mdlp), '')

samplePrior: Sample prior (min = 1.0) (java.lang.Double) [default:1.0]
structurePrior: Structure prior coefficient (min = 1.0) (java.lang.Double) [default:1.0]
faithfulnessAssumed: Yes if (one edge) faithfulness should be assumed (java.lang.Boolean) [default:true]
symmetricFirstStep: Yes if the first step step for FGES should do scoring for both X->Y and Y->X (java.lang.Boolean) [default:false]
maxDegree: The maximum degree of the graph (min = -1) (java.lang.Integer) [default:100]
verbose: Yes if verbose output should be printed or logged (java.lang.Boolean) [default:true]
numberResampling: The number of resampling iterations (min = 0) (java.lang.Integer) [default:0]
resampleSize: The resample size (min = 1) (java.lang.Integer) [default:1]
resamplingWithReplacement: Yes, if resampling with replacement (bootstrapping) (java.lang.Boolean) [default:true]
resamplingEnsemble: Ensemble method: Preserved (0), Highest (1), Majority (2) (java.lang.Integer) [default:1]
edges ['class --- VisITedR

In [23]:

dataset_path = '../server/{}.csv'.format(dataset_name)
data = pd.read_csv(dataset_path)
mdlp = num2cate_fit(data)



samples_path = os.path.join(store_path, '{}_samples.json'.format(dataset_name))
samplesInit = pd.read_json(samples_path)

# set name
model_path = dataset_name+"_"+model_name
model_gene = ModelGene(model_path)

# train model
model, encoder, score = model_gene.fit_model( num2cate_transform(data, mdlp) )



# post process
samples = num2cate_transform(samplesInit, mdlp)
# model predict proba
x_samples, _ = encoder.transform(samples)
y_samples = model.predict(x_samples)
y_probs = model.predict_proba(x_samples)

score

/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wangqianwen/Documents/git_repo/disc

0.9375

In [50]:
def predict_post(x_samples, model, theta, samples, constraints=[['gender', 'M']]):
    y_samples = model.predict(x_samples)
    y_probs = model.predict_proba(x_samples)
    y_post_samples = []
    for idx, prob in enumerate( y_probs):
        # if in the reject option space, modify label
        if max(prob[1], 1- prob[1])<theta and all([samples.iloc[idx][constraint[0]]==constraint[1] for constraint in constraints]):
            y_post_samples.append(1)
        # else, remain the same
        else:
            y_post_samples.append(y_samples[idx])
    return y_post_samples

x, y = encoder.transform(num2cate_transform(data, mdlp))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
y_post_test = predict_post(x_test, model, theta, data, '')
score = accuracy_score(y_test, y_post_test)
score

/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/server/model/data_encoder.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cats[column] = encoder.transform(cats[column])


0.9166666666666666

In [58]:
y_samples_post = predict_post(x_samples, model, theta, samples, [['gender', 'M']])

#  concate 
num_samples = samplesInit.copy()
num_samples['class'] = pd.Series(np.asarray(y_samples_post), index= samples.index) 

cate_samples = samples.copy()
cate_samples['class'] = pd.Series(np.asarray(y_samples_post), index= samples.index) 

cate_samples.insert(loc=0, column='id', value= cate_samples.index)
num_samples.insert(loc=0, column='id', value= num_samples.index)

dataOut = pd.concat([num_samples,cate_samples])

samples_path = os.path.join(store_path, '{}_{}_post_samples.json'.format(dataset_name, model_name))
dataOut.to_json(samples_path, orient='records')
samples_path = os.path.join(store_path, '{}_{}_post_samples.csv'.format(dataset_name, model_name))
dataOut.to_csv(samples_path)

In [59]:
y_samples_post = predict_post(x_samples, model, theta, samples, [['gender', 'M'], ['StudentAbsenceDays', 'Above-7']])

#  concate 
num_samples = samplesInit.copy()
num_samples['class'] = pd.Series(np.asarray(y_samples_post), index= samples.index) 

cate_samples = samples.copy()
cate_samples['class'] = pd.Series(np.asarray(y_samples_post), index= samples.index) 

cate_samples.insert(loc=0, column='id', value= cate_samples.index)
num_samples.insert(loc=0, column='id', value= num_samples.index)

dataOut = pd.concat([num_samples,cate_samples])

samples_path = os.path.join(store_path, '{}_{}_post2_samples.json'.format(dataset_name, model_name))
dataOut.to_json(samples_path, orient='records')
samples_path = os.path.join(store_path, '{}_{}_post2_samples.csv'.format(dataset_name, model_name))
dataOut.to_csv(samples_path)

In [60]:
x, y = encoder.transform(num2cate_transform(data, mdlp))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
y_post_test = predict_post(x_test, model, theta, data, [['gender', 'M'], ['StudentAbsenceDays', 'Above-7']])
score = accuracy_score(y_test, y_post_test)

/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/myenv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/wangqianwen/Documents/git_repo/discrimination_in_ML/py/server/model/data_encoder.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cats[column] = encoder.transform(cats[column])


In [61]:
score

0.9166666666666666